# Transfer Learning

In the previous notebook we learned how to create custom neural networks using the gluon API. This enabled the creation of custom neural networks, but there is one severly limiting factor no matter the network chosen: a lack of training data.

While the power and popularity of neural networks has increased dramatically, the amount of available training data hasn't quite matched pace. This is unfortunate since even the best designed neural network is meaningless without data to train against (excluding some recent developments in reinforcement learning, but we won't talk about that here). While the general availability of labeled data may be small, there are some prominent efforts to create large amounts of labeled data. Since we are focused on images here, the prominent dataset to mention is imagenet. Imagenet is the largest (to the author's knowledge) available labeled image dataset, and models trained on imagenet can be obtained for free and repurposed.

This notebook will focus on an idea called transfer learning. The concept is fairly simple. A model trained on one dataset may also be good at understanding another dataset. The power of transfer learning is hard to overstate. While the imagenet dataset is quite impressive, it fails to encompass the entirety of what may be desired of labeled image data. However, if a model trained on imagenet data can be repurposed with great effect, then the value of imagenet is drastically incrased.

## Goal
In this notebook we will take a model trained on imagenet and repurose the model to perform the same face detection tasks that we built in the previous notebook. While the gluon API will be used, the symbol API will also be leveraged to produce the final output.

## Load the necessary libraries

In [1]:
import keras
from keras import applications
from keras.preprocessing import image
import numpy as np
import os
from skimage import transform
from skimage import io
from scipy.misc import imresize
from glob import glob
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: No module named keras

Keras uses the hdf5 file structure to store it's models. We need to install this package to be able to load a trained model.

In [3]:
! source /home/ec2-user/src/anaconda3/bin/activate & sudo pip install h5py
print('finished')

finished


## Where are the images?
Tell Python where the images are located that we will work with (this assumes that you have not removed the images downloaded in the previous notebook).

In [4]:
base = os.getcwd() # If you manually changed your working directory this will not work
old = 'faces' #specify a location to store the images we will work with
new = 'bigger_faces'
old_loc = os.path.join(base, old)
new_loc = os.path.join(base, new)
if not os.path.isdir(new_loc):
    os.mkdir(new_loc)

if not os.path.isdir(os.path.join(new_loc, 'target_ariel')):
    os.mkdir(os.path.join(new_loc, 'target_ariel'))

positive = os.path.join(new_loc, 'target_ariel', 'positive')
negative = os.path.join(new_loc, 'target_ariel', 'negative')
if not os.path.isdir(positive):
    os.mkdir(positive) #folder to hold all of the pictures of Ariel
    
if not os.path.isdir(negative):
    os.mkdir(negative) #folder to hold pictures of everyone else    
    
loc = os.path.join(base, old)
old_root = os.path.join(loc, 'target_ariel')
positive = os.path.join(old_root, 'positive')
negative = os.path.join(old_root, 'negative')
image_root = old_root.replace(old, new)

In [5]:
#resize the images so that we can use them with a different model
files = glob(os.path.join(old_root, '*/*.jpg'))
num_images = len(files)
for i, file_ in enumerate(files):
    img = io.imread(file_)
    new_loc = file_.replace(old, new)
    new_image = transform.resize(img, (300, 300), mode='constant')
    io.imsave(new_loc, new_image)
    print('\rCompleted image number: {}'.format(i), end='')
print('\nfinished')

/usr/lib64/python3.4/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


Completed image number: 1053
finished


## Get a model from keras

In [6]:
# If this is your first time running the code, this will take some time to download the parameters
inception = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(300, 300, 3))
print('finished downloading')

finished downloading


## Decapitate the model
It sounds rough, but we generally need to decapitate a pretrained model to make use of it. Recall from the model we made in the last notebook, the top layer had one output that was designed to determine if an image was of Ariel or not. This top layer is only useful for the exact problem the model was trained for. But, the same does not hold true for the layers before the top layer!

### Stand on the shoulders of giants
Think back to the introduction to convolutional layers. The convolutional layer looks for specific patterns in an image. It turns out that in order to understand images, the features of importance can be realtively general amongst image-based tasks. Further, the only layer of a neural network that is not general is the last layer (the one we will remove). This is how we can leverage the immense amount of training that went into making the imagenet model for something that we care about.

Thankfully keras has us in mind, and the above command decapitated the model by us simply setting the include top flag to False.

### Create a classifier
Here we take the neural network output and we build a simple logistic regression classifier on it. While this isn't the most powerful model on it's own, but with the powerful neural network in front, we'll give it a shot.

In [7]:
model = LogisticRegression(class_weight='balanced')

## Get the image information
We now need to setup a function that will get the image information and pass it through our network.

In [8]:
gen = image.ImageDataGenerator()
batch_size = 50
flow = gen.flow_from_directory(image_root, target_size=(300, 300), color_mode='rgb', class_mode='binary',
                              shuffle=False, batch_size=batch_size)

Found 1054 images belonging to 2 classes.


## Determine the shape of the network output

In [9]:
inception.layers[-1].output.shape

TensorShape([Dimension(None), Dimension(8), Dimension(8), Dimension(2048)])

## Gather the features

In [14]:
#make an array to hold the neural network output
net_output = np.zeros((num_images, 8*8*2048))
labels = np.zeros((num_images,))
batches = int(np.ceil(num_images/batch_size))
low = 0
high = 0

for i, pic in enumerate(range(batches)):
    data = flow.next()
    num_samples = data[1].shape[0]
    high = low + num_samples
    output = inception.predict(data[0])
    output = output.reshape((num_samples, -1))
    net_output[low:high, :] = output
    labels[low:high] = data[1]

    low = high
    print('\rFinished batch number: {}'.format(i), end='')
    
print('\nComplete!')

Finished batch number: 21
Complete!


## Train the model

In [15]:
model.fit(net_output, labels)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

## Make a prediction
Here we will load an image and make a prediction on it

In [24]:
positives = positive.replace(old, new)

pics = glob(os.path.join(positives, '*'))
img = io.imread(pics[3])
img = np.array(img)
img = img.reshape((1, 300, 300, 3))

output = inception.predict(img)
output = output.flatten()
output = output.reshape((1, -1))
output = model.predict(output)
print(output)

[ 1.]
